## Generative vs Discriminative

In ML, one typically makes distinction between generative vs discriminative classifiers:
- Generative classifiers estimate class density, prior, and evidence (i.e. terms on the RHS). Example is Quadratic Discriminant Analysis.
- Discriminative classifiers directly estimate posterior probability. Examples are k-NN, decision tree, random forest, etc.

Although kernel density estimate can be used to approximate the class density, the data might have too few points and too many dimensions that make such kernel density estimate unreliable. In **quadratic discriminant analysis (QDA)**, we approximate each class in the training data by a single Gaussian (estimate the mean and covariance matrix). Then we apply Bayes theorem to compute the posterior probability. Note that we also need to estimate the prior (can be easily done by counting # of classes). The shape of the decision boundary can be found via Bayes Theorem as follows.

$$\begin{align*} p(1|x) & = p(2|x) \\
    \frac{p(x|1) p(1)}{p(x)} & = \frac{p(x|2) p(2)}{p(x)} \\
    c_1 \exp[\frac{-1}{2} (x - \mu_1)^T \Sigma_1^{-1} (x - \mu_1)] p(1) & = c_2 \exp[\frac{-1}{2}(x - \mu_2)^T \Sigma_2^{-1} (x - \mu_2)] p(2) \\
    \ln\frac{c_1 p_1}{c_2 p_2} - \frac{1}{2} (x - \mu_1)^T \Sigma_1^{-1} (x - \mu_1) + \frac{1}{2} (x - \mu_2)^T \Sigma_2^{-1} (x - \mu_2) & = 0
\end{align*}
$$

Now, the first term is a scalar constant, and therefore the equation is quadratic in $x$. This suggests that the decision boundaries of QDA are **quadrics**, which is an umbrella term for ellipse, hyperbola, parabola, and parallel lines.

In [12]:
df.describe()

,height,weight,games,victories,ties,defeats,goals,yellowCards,yellowReds,redCards,rater1,rater2,refNum,refCountry,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
count,145765.000000,143785.000000,146028.000000,146028.000000,146028.000000,146028.000000,146028.000000,146028.000000,146028.000000,146028.000000,124621.000000,124621.000000,146028.000000,146028.000000,145865.000000,1.458650e+05,1.458650e+05,145865.000000,1.458650e+05,145865.000000
mean,181.935938,76.075662,2.921166,1.278344,0.708241,0.934581,0.338058,0.385364,0.011381,0.012559,0.264255,0.302862,1534.827444,29.642842,0.346276,1.969741e+04,6.310849e-04,0.452026,2.044023e+04,0.002994
std,6.738726,7.140906,3.413633,1.790725,1.116793,1.383059,0.906481,0.795333,0.107931,0.112889,0.295382,0.293020,918.736625,27.496189,0.032246,1.271262e+05,4.735857e-03,0.217469,1.306157e+05,0.019723
min,161.000000,54.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,-0.047254,2.000000e+00,2.235373e-07,-1.375000,2.000000e+00,0.000001
25%,177.000000,71.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,641.000000,7.000000,0.334684,1.785000e+03,5.454025e-05,0.336101,1.897000e+03,0.000225
50%,182.000000,76.000000,2.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.250000,1604.000000,21.000000,0.336628,2.882000e+03,1.508847e-04,0.356446,3.011000e+03,0.000586
75%,187.000000,81.000000,3.000000,2.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.250000,0.500000,2345.000000,44.000000,0.369894,7.749000e+03,2.294896e-04,0.588297,7.974000e+03,0.001002
max,203.000000,100.000000,47.000000,29.000000,14.000000,18.000000,23.000000,14.000000,3.000000,2.000000,1.000000,1.000000,3147.000000,161.000000,0.573793,1.975803e+06,2.862871e-01,1.800000,2.029548e+06,1.060660


## Multivariate Normal

A multivariate Gaussian has two parameters: mean and covariance, where the covariance is defined as

$$ \begin{split}
Cov(X) & := \mathbb{E}[(X - \mathbb{E}[X]) \, (X - \mathbb{E}[X])^T] \\
    & = \mathbb{E}[XX^T - X \mathbb{E}[X^T] - E[X] X^T + E[X] E[X^T]] \\
    & = \mathbb{E}[XX^T] - \mathbb{E}[X] \mathbb{E}[X^T]
    \end{split}
$$

- $X$ is a vector $p \times 1$ of random variables
- $\mathbb{E}[\mathbb{E}[X]] = \mathbb{E}[X]$ because expectation value of a random variable is a constant, and expectation value of a constant is the constant itself.
- It follows that covariance of an affinely-transposed random variable is $Cov(AX) = A Cov(X) A^T$ for some matrix A.
- The covariance matrix has dimension $p \times p$.

Empirical estimate of covariance is given by

$$ \frac{1}{n-1} S = \frac{1}{n-1}XCC^T X^T = \frac{1}{n-1}XCX^T$$

where S is the scatter matrix and C is the centering matrix，with $CC^T = C$. Once the data is centered once, nothing happens if we center it again $C = I - \frac{1}{n} 1 * 1^T$; here $1$ is the column vector of ones. The isocountours of a multivariate Gaussian are shown below for negative cross-correlation (left), stronger negative cross-correlation, and positive cross-correlation with higher spread at the 2nd dimension.

<img src="img/covariance.png" alt="Drawing" style="width: 600px;"/>

The diagonal elements has the variance of each dimension. The off-diagonal elements indicate how much linear association is between random variable along different dimensions. The **normal distribution** is defined as

$$p(x) = \frac{1}{(2\pi)^{p/2} |\Sigma|^{1/2}} \exp[-\frac{1}{2}(x - \mu)^T \Sigma^{-1} (x - \mu)]$$

Points worth noting:
- The inverse covariance matrix $\Sigma^{-1}$ is also called the precision matrix: the larger the covariance/spread, the "better" the precision is.
- $\mu$ is the mean vector of the data
- marginal of a normal distribution is normal
- conditional of a normal distribution is normal
- normal marginal doesn't necessarily give normal joint distribution

See https://juanitorduz.github.io/multivariate_normal/

In [ ]:
df.age = 